In [62]:
#library umum
import numpy as np
import pandas as pd
import gdown
import os 
from random import sample
import random
random.seed(43)
import shutil
from shutil import copyfile
from tqdm.notebook import tqdm
import uuid

In [63]:
#library image processing
import cv2
from PIL import Image
import skimage
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

In [339]:
# libraray untuk pembangunan model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Data Preparation
menyiapkan data untuk dapat digunakan dalam pelatihan
disini kami menyiapkan data dengan mensetting dan melabeli data agar lebih jelas dan mudah unutk diinterpretasikan

In [65]:
url = "https://drive.google.com/uc?id=1TbHce2SU-W_Fr2hMjN9E_WhC_1R9M182"
output = 'Penyisihan_Hology_Data_Mining.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output)
os.remove(output)

Downloading...
From: https://drive.google.com/uc?id=1TbHce2SU-W_Fr2hMjN9E_WhC_1R9M182
To: c:\Users\User\Documents\hehehe\penyisihan_hology\Penyisihan_Hology_Data_Mining.zip
100%|██████████| 10.2M/10.2M [00:03<00:00, 2.73MB/s]


In [66]:
train_df = pd.read_csv('Penyisihan Hology Data Mining/train.csv')
train_df.head()

,id,jenis,warna
0,1,1,1
1,2,0,2
2,3,1,3
3,4,1,1
4,5,0,4


In [67]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      777 non-null    int64
 1   jenis   777 non-null    int64
 2   warna   777 non-null    int64
dtypes: int64(3)
memory usage: 18.3 KB


In [68]:
train_df['jenis_1'] = train_df['jenis'].apply(lambda x: 'Kaos' if x == 0 else 'Hoodie')

In [69]:
train_df['warna_1'] = train_df['warna'].apply(lambda x: 
                                              'merah' if x == 0 else
                                              'kuning' if x == 1 else
                                              'biru' if x == 2 else
                                              'hitam' if x == 3 else
                                              'putih')

In [70]:
train_df.head(20)

,id,jenis,warna,jenis_1,warna_1
0,1,1,1,Hoodie,kuning
1,2,0,2,Kaos,biru
2,3,1,3,Hoodie,hitam
3,4,1,1,Hoodie,kuning
4,5,0,4,Kaos,putih
5,6,1,3,Hoodie,hitam
6,7,0,4,Kaos,putih
7,8,1,2,Hoodie,biru
8,9,1,3,Hoodie,hitam
9,10,1,0,Hoodie,merah


In [71]:
# melihat persebaran data
train_df['jenis_1'].value_counts()

jenis_1
Kaos      476
Hoodie    301
Name: count, dtype: int64

In [72]:
train_df['warna_1'].value_counts()

warna_1
hitam     234
biru      162
putih     140
kuning    125
merah     116
Name: count, dtype: int64

In [73]:
train_df.groupby(['jenis_1', 'warna_1']).agg({
    'id' : 'nunique'
})
#terlihat bahwa persebaran data tidak seimbang antara satu jenis dengan jenis yang lain

id
jenis_1 warna_1     
Hoodie  biru      59
        hitam    103
        kuning    54
        merah     36
        putih     49
Kaos    biru     103
        hitam    131
        kuning    71
        merah     80
        putih     91

mengambil path dan mengubah nama dari setiap file gambar pada folder train agar dapat lebih mudah untuk didentifikasi yang nantinya akan digunakan untuk data augmentation

In [74]:
file_path = "Penyisihan Hology Data Mining/train/"

In [75]:
img_name = list()

In [76]:
# mengambil nama file gambar pada data folder train
for path, subdir, files in os.walk(file_path):
    for name in files:
        img_name.append(name) #menyimpan hasil ke variabel list img_name

print (img_name)

['1.jpg', '10.jpg', '100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '108.jpg', '109.jpg', '11.jpg', '110.jpg', '111.jpg', '112.jpg', '113.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '118.png', '119.jpg', '12.jpg', '120.jpg', '121.jpg', '122.jpg', '123.jpg', '124.jpg', '125.jpg', '126.jpg', '127.png', '128.jpg', '129.jpg', '13.jpg', '130.jpg', '131.jpg', '132.jpg', '133.jpg', '134.jpg', '135.jpg', '136.jpg', '137.jpg', '138.png', '139.jpg', '14.jpg', '140.jpg', '141.jpg', '142.jpg', '143.jpg', '144.jpg', '145.jpg', '146.jpg', '147.jpg', '148.jpg', '149.jpg', '15.jpg', '150.jpg', '151.jpg', '152.jpg', '153.jpg', '154.jpg', '155.jpg', '156.jpg', '157.jpg', '158.jpg', '159.jpg', '16.jpg', '160.jpg', '161.jpg', '162.jpg', '163.jpg', '164.jpg', '165.jpg', '166.jpg', '167.jpg', '168.jpg', '169.jpg', '17.jpg', '170.jpg', '171.jpg', '172.jpg', '173.jpg', '174.jpg', '175.jpg', '176.jpg', '177.jpg', '178.jpg', '179.jpg', '18.png', '180.jpg', '181.jpg'

In [77]:
# membuat dataframe yang baru yang menampung hasil dari extract nama sebelumnya di filedf
file_df = pd.DataFrame({'filename': img_name})

#membuat column id untuk setiap nama file yang sudah didapatkan karena id sama dengan nama nomer nama file gambar yang ada
file_df['id'] = file_df['filename'].apply(lambda x: int(x.split('.')[0]))
file_df.head()

,filename,id
0,1.jpg,1
1,10.jpg,10
2,100.jpg,100
3,101.jpg,101
4,102.jpg,102


In [78]:
#menggabungkan train_df dengan file_df(dataframe nama file) sesuai dengan idnya
#hal ini dapat mengotomatisasi proses penambahan column filename ke train_df
train_df = pd.merge(train_df, file_df, on='id', how='inner')
train_df.head(10)

,id,jenis,warna,jenis_1,warna_1,filename
0,1,1,1,Hoodie,kuning,1.jpg
1,2,0,2,Kaos,biru,2.jpg
2,3,1,3,Hoodie,hitam,3.jpg
3,4,1,1,Hoodie,kuning,4.jpg
4,5,0,4,Kaos,putih,5.jpg
5,6,1,3,Hoodie,hitam,6.jpg
6,7,0,4,Kaos,putih,7.jpg
7,8,1,2,Hoodie,biru,8.jpg
8,9,1,3,Hoodie,hitam,9.jpg
9,10,1,0,Hoodie,merah,10.jpg


In [79]:
df = train_df.drop(columns=['id', 'jenis', 'warna'])
df.head()

,jenis_1,warna_1,filename
0,Hoodie,kuning,1.jpg
1,Kaos,biru,2.jpg
2,Hoodie,hitam,3.jpg
3,Hoodie,kuning,4.jpg
4,Kaos,putih,5.jpg


# Data Augmentation
disini kami akan menambah kekurangan jumlah data pada beberapa kategori gambar yang ada agar data dapat seimbang atau tidak berbeda jauh dengan data yang lainnya

Daftar fungsi untuk melakukan data augmentation

In [80]:
# fungsi untuk merotasi gambar berlawanan arah jarum jam
def anticlockwise_rotation(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    sudut = random.randint(0, 180)
    return rotate(img, sudut)

In [81]:
# fungsi untuk merotasi gambar searah dengan jarum jam
def clockwise_rotation(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    sudut = random.randint(0, 180)
    return rotate(img, -sudut)

In [82]:
# Membuat fungsi untuk membalik gambar secara vertikal dari atas ke bawah
def flip_up_down(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    return np.flipud(img)

In [83]:
# fungsi untuk memberikanefek peningkatan kecerahan pada gambar
def  add_brightness(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = adjust_gamma(img, gamma=0.5, gain=1)
    return img

In [84]:
# fungsi untuk membuat blur gambar
def blur_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    return cv2.GaussianBlur(img, (9,9), 0)

In [85]:
# fungsi untuk menggeser gambar beberapa pixel
def sheared(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    transform = AffineTransform(shear=0.2)
    shear_image = warp(img, transform, mode="wrap")
    return shear_image

In [86]:
# fungsi untuk melakukan pergeseran melengkung pada gambar

def warp_shift(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    transform = AffineTransform(translation=(0, 40))
    warp_image = warp(img, transform, mode='wrap')
    return warp_image

In [87]:
#variabel transformasi yang akan menyimpan semua fungsi data augmentasi 
transformations = {
    'rotate anticlockwise': anticlockwise_rotation,
    'rotate clockwise' : clockwise_rotation,
    'warp shift':warp_shift,
    'blurring image': blur_image,
    'add brightness': add_brightness,
    'flip up down': flip_up_down,
    'shear image': sheared
}

In [88]:
def augmented(count, images, jenis, warna):
    master_path = "Penyisihan Hology Data Mining/train/"
    i = 1 # iterasi
    df1 = []
    for i in range(count):
        preimage = random.choice(images) # memilih gambar yang akan di augmented

        try:
            original_image = io.imread(os.path.join(file_path, preimage)) 
            transformed_image = None
            transformation_count = random.randint(1, len(transformations)) # memilih angka acak berapa banyak teknik transformasi yang akan dipake dari variabel transformasi
            
            #melakukan augmentasi gambar pada gambar yang sudah dipilih di orginial image
            for _ in range(transformation_count):
                key = random.choice(list(transformations))
                transformed_image = transformations[key](original_image) # menerapkan augmentasi gambar dari pemilihan fungsi augmentasi yang sebelumnya telah dibuat
        
            unique_id = uuid.uuid4().hex #generates random unique id for filename of new image
            new_image_path = f"{master_path}/T_{unique_id}_{preimage}" #memberikan path untuk gambar baru yang sudah di augmentasi (seperti memberi alamat untuk penyimpanan)
            transformed_image = img_as_ubyte(transformed_image) # mengonversi gambar ke format byte
            cv2.imwrite(new_image_path, transformed_image) # simpan gamber pada path sudah ditentukan

            # menambahkan juga data pada table secara otomatis
            new_row = {'jenis_1':jenis, 'warna_1':warna, 'filename':f'T_{unique_id}_{preimage}'}
            df1.append(new_row)
        # jika gambar tidak dapat dibuka maka akan diskip dan diganti dengan gambar selanjutnya
        except ValueError as e:
            print(f'could not read the {preimage} : {e} hence skipping it.')
    df1 = pd.DataFrame(df1)
    return df1

In [89]:
jenis = list(df['jenis_1'].unique())
warna = list(df['warna_1'].unique())

In [90]:
print (jenis)

['Hoodie', 'Kaos']


In [91]:
coba1 = df[df['jenis_1'] == 'Hoodie']['warna_1'].value_counts()
print (coba1.max())

103


In [92]:
max_count = 0
for x in jenis:
    for y in warna:
        if df[(df['jenis_1'] == x) & (df['warna_1'] == y)]['filename'].count() > max_count:
            max_count = df[(df['jenis_1'] == x) & (df['warna_1'] == y)]['filename'].count()

In [93]:
for x in jenis:
    for y in warna:
        minus = df[(df['jenis_1'] == x) & (df['warna_1'] == y)]['filename'].count()
        if minus < max_count:
            total = max_count - minus
            images = list(df[(df['jenis_1'] == x) & (df['warna_1'] == y)]['filename'])
            df1 = augmented(total, images, x, y)
            df = pd.concat([df, df1], ignore_index=True)

In [94]:
df.groupby(['jenis_1', 'warna_1']).agg({
    'filename':'nunique'
})

filename
jenis_1 warna_1          
Hoodie  biru          131
        hitam         131
        kuning        131
        merah         131
        putih         131
Kaos    biru          131
        hitam         131
        kuning        131
        merah         131
        putih         131

In [95]:
df.tail()

,jenis_1,warna_1,filename
1305,Kaos,merah,T_3fc43cc7b9684a0daffdd109f3f90f05_403.png
1306,Kaos,merah,T_f831d001a1e44180b5c31342fa26407b_550.jpg
1307,Kaos,merah,T_4d5866bd65cf49d1a6e6f0f4eaeaca97_539.jpg
1308,Kaos,merah,T_304b6afece9a47b283516471cdbeb195_383.jpg
1309,Kaos,merah,T_6a7a14bff23b4926849ef3aabf77999c_663.jpg


#Multi hot encoding

In [96]:
# memberikan alamat lengkap setiap file gambar sesuai dengan filename dalam dataframe
df['path'] = df['filename'].apply(lambda x: f'{file_path}{x}')

In [97]:
# menghapus columns yang sudah tidak dipakai lagi yaitu column filename
df = df.drop(columns=['filename'])

In [98]:
df['jenis+warna'] = df.apply(lambda x: [x['jenis_1'], x['warna_1']], axis=1)

In [99]:
df.head(10)

,jenis_1,warna_1,path,jenis+warna
0,Hoodie,kuning,Penyisihan Hology Data Mining/train/1.jpg,"[Hoodie, kuning]"
1,Kaos,biru,Penyisihan Hology Data Mining/train/2.jpg,"[Kaos, biru]"
2,Hoodie,hitam,Penyisihan Hology Data Mining/train/3.jpg,"[Hoodie, hitam]"
3,Hoodie,kuning,Penyisihan Hology Data Mining/train/4.jpg,"[Hoodie, kuning]"
4,Kaos,putih,Penyisihan Hology Data Mining/train/5.jpg,"[Kaos, putih]"
5,Hoodie,hitam,Penyisihan Hology Data Mining/train/6.jpg,"[Hoodie, hitam]"
6,Kaos,putih,Penyisihan Hology Data Mining/train/7.jpg,"[Kaos, putih]"
7,Hoodie,biru,Penyisihan Hology Data Mining/train/8.jpg,"[Hoodie, biru]"
8,Hoodie,hitam,Penyisihan Hology Data Mining/train/9.jpg,"[Hoodie, hitam]"
9,Hoodie,merah,Penyisihan Hology Data Mining/train/10.jpg,"[Hoodie, merah]"


In [100]:
# melakukan multi hot encoding pada data multi label menggunakan MultiLabelBinarizer()
mlb = MultiLabelBinarizer()
multi_label = mlb.fit_transform(df['jenis+warna'])

In [101]:
label_class = mlb.classes_

In [102]:
df_multi_label = pd.DataFrame(multi_label, columns=label_class)
df_multi = pd.concat([df['path'], df_multi_label], axis=1)

In [103]:
df_multi.head()

,path,Hoodie,Kaos,biru,hitam,kuning,merah,putih
0,Penyisihan Hology Data Mining/train/1.jpg,1,0,0,0,1,0,0
1,Penyisihan Hology Data Mining/train/2.jpg,0,1,1,0,0,0,0
2,Penyisihan Hology Data Mining/train/3.jpg,1,0,0,1,0,0,0
3,Penyisihan Hology Data Mining/train/4.jpg,1,0,0,0,1,0,0
4,Penyisihan Hology Data Mining/train/5.jpg,0,1,0,0,0,0,1


# Build Model

In [104]:
label_columns = ['Hoodie', 'Kaos', 'biru', 'hitam', 'kuning', 'merah', 'putih']
multi_label = mlb.fit_transform(df_multi[label_columns].values.tolist())

In [368]:
# kami menggunakan banyak kombinasi pelatihan yang diatur oleh random state dimulai dari state 46
traindf, valdf, train_labels, val_labels = train_test_split(
    df_multi, multi_label, test_size=0.2, random_state=58, stratify=multi_label
)

In [321]:
print(traindf)

                                                   path  Hoodie  Kaos  biru  \
501         Penyisihan Hology Data Mining/train/502.jpg       1     0     1   
1290  Penyisihan Hology Data Mining/train/T_c1e246c4...       0     1     0   
475         Penyisihan Hology Data Mining/train/476.png       0     1     0   
543         Penyisihan Hology Data Mining/train/544.jpg       0     1     1   
511         Penyisihan Hology Data Mining/train/512.jpg       1     0     0   
...                                                 ...     ...   ...   ...   
723         Penyisihan Hology Data Mining/train/724.jpg       0     1     0   
694         Penyisihan Hology Data Mining/train/695.jpg       0     1     0   
736         Penyisihan Hology Data Mining/train/737.jpg       1     0     1   
589         Penyisihan Hology Data Mining/train/590.jpg       0     1     0   
949   Penyisihan Hology Data Mining/train/T_54b3144f...       1     0     0   

      hitam  kuning  merah  putih  
501       0    

In [322]:
def exact_match_ratio(y_true, y_pred):
    # Convert y_pred to binary values (0 or 1) for thresholded predictions
    y_pred = tf.round(y_pred)
    # Ensure both y_true and y_pred are of the same dtype
    y_true = tf.cast(y_true, tf.float32)
    # Check if each row (instance) exactly matches the corresponding row in y_true
    exact_match = tf.reduce_all(tf.equal(y_true, y_pred), axis=1)
    # Calculate mean of exact matches (1 if matches, 0 otherwise)
    return tf.reduce_mean(tf.cast(exact_match, tf.float32))

In [324]:
datagen = ImageDataGenerator(rescale=1./255)

In [369]:
train_gen = datagen.flow_from_dataframe(
    traindf,
    x_col = 'path',
    y_col = label_columns,
    target_size = (128, 128),
    color_mode = 'rgb',
    class_mode = 'raw',
    batch_size = 16
)

Found 1048 validated image filenames.


In [370]:
val_gen = datagen.flow_from_dataframe(
    valdf,
    x_col = 'path',
    y_col = label_columns,
    target_size = (128, 128),
    color_mode = 'rgb',
    class_mode = 'raw',
    batch_size = 16
)

Found 262 validated image filenames.


In [362]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
    Conv2D(256, (3,3)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),

    Conv2D(512, (3,3), activation='relu'),
    BatchNormalization(),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    Dropout(0.3),
    
    Dense(7, activation='sigmoid')
])

In [276]:
initial_learning_rate = 1e-4
decay_rate = 1
decay_steps = 1000

lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True
)

In [364]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics = [exact_match_ratio]) #'binary_accuracy'

In [348]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5,  min_lr=0.000001),
    #EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
]

In [371]:
history = model.fit(train_gen, validation_data=val_gen,epochs=30, batch_size=32,verbose=1, callbacks=callbacks)

Epoch 1/30


c:\Users\User\miniconda3\envs\project\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 26s 398ms/step - exact_match_ratio: 0.9173 - loss: 0.0647 - val_exact_match_ratio: 0.9632 - val_loss: 0.0296 - learning_rate: 1.0000e-06
Epoch 2/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 30s 445ms/step - exact_match_ratio: 0.9173 - loss: 0.0589 - val_exact_match_ratio: 0.9632 - val_loss: 0.0297 - learning_rate: 1.0000e-06
Epoch 3/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 31s 463ms/step - exact_match_ratio: 0.9167 - loss: 0.0709 - val_exact_match_ratio: 0.9632 - val_loss: 0.0298 - learning_rate: 1.0000e-06
Epoch 4/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 32s 482ms/step - exact_match_ratio: 0.9129 - loss: 0.0627 - val_exact_match_ratio: 0.9596 - val_loss: 0.0298 - learning_rate: 1.0000e-06
Epoch 5/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 30s 448ms/step - exact_match_ratio: 0.9042 - loss: 0.0678 - val_exact_match_ratio: 0.9632 - val_loss: 0.0298 - learning_rate: 1.0000e-06
Epoch 6/30
66/66 ━━━━━━━━━━━━━━━━━━━━ 28s 415ms/step - exact_match_ratio: 0.9044 - loss: 0.0673 - val_exact_match_ratio: 0.9571 - val_lo

In [166]:
model.save('saved_model_new/my_model4_final.keras')

# Data Preparation Test
disini kami menyiapakan format data yang sesuai untuk digunakan dalam pengujian model proses hampir sama dengan dataframe.

In [205]:
test_file_path = 'Penyisihan Hology Data Mining/test/'
test_df = pd.read_csv('Penyisihan Hology Data Mining/sample_submission.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      334 non-null    int64
 1   jenis   334 non-null    int64
 2   warna   334 non-null    int64
dtypes: int64(3)
memory usage: 8.0 KB


In [206]:
test_df.head(20)

,id,jenis,warna
0,778,0,0
1,779,0,0
2,780,0,0
3,781,0,0
4,782,0,0
5,783,0,0
6,784,0,0
7,785,0,0
8,786,0,0
9,787,0,0


In [208]:
# membuang bagian yang tidak dipakai lagi
test_df.drop(columns=['jenis', 'warna'], inplace=True)

In [209]:
test_df.head()

,id
0,778
1,779
2,780
3,781
4,782


In [210]:
test_img_name = list()

In [211]:
# mengambil nama file gambar pada data folder test
for path, subdir, files in os.walk(test_file_path):
    for name in files:
        test_img_name.append(name) #menyimpan hasil ke variabel list test_img_name

In [215]:
print(test_img_name)

['1000.jpg', '1001.jpg', '1002.jpg', '1003.png', '1004.jpg', '1005.jpg', '1006.png', '1007.jpg', '1008.png', '1009.jpg', '1010.jpg', '1011.jpg', '1012.jpg', '1013.jpg', '1014.jpg', '1015.jpg', '1016.jpg', '1017.jpg', '1018.jpg', '1019.jpg', '1020.jpg', '1021.jpg', '1022.png', '1023.jpg', '1024.jpg', '1025.jpg', '1026.png', '1027.jpg', '1028.jpg', '1029.jpg', '1030.png', '1031.png', '1032.jpg', '1033.jpg', '1034.jpg', '1035.jpg', '1036.png', '1037.jpg', '1038.jpg', '1039.jpg', '1040.jpg', '1041.png', '1042.png', '1043.jpg', '1044.jpg', '1045.jpg', '1046.jpg', '1047.jpg', '1048.jpg', '1049.jpg', '1050.jpg', '1051.jpg', '1052.jpg', '1053.png', '1054.jpg', '1055.jpg', '1056.png', '1057.jpg', '1058.jpg', '1059.jpg', '1060.jpg', '1061.png', '1062.jpg', '1063.jpg', '1064.jpg', '1065.png', '1066.png', '1067.jpg', '1068.jpg', '1069.jpg', '1070.jpg', '1071.jpg', '1072.png', '1073.jpg', '1074.jpg', '1075.jpg', '1076.png', '1077.jpg', '1078.jpg', '1079.jpg', '1080.jpg', '1081.jpg', '1082.jpg', '10

In [216]:
# membuat dataframe yang baru yang menampung hasil dari extract nama sebelumnya di filedf
test_file_df = pd.DataFrame({'filename': test_img_name})

# membuat column id untuk setiap nama file yang sudah didapatkan karena id sama dengan nama nomer nama file gambar yang ada
test_file_df['id'] = test_file_df['filename'].apply(lambda x: int(x.split('.')[0]))

In [217]:
test_file_df.head(10)

,filename,id
0,1000.jpg,1000
1,1001.jpg,1001
2,1002.jpg,1002
3,1003.png,1003
4,1004.jpg,1004
5,1005.jpg,1005
6,1006.png,1006
7,1007.jpg,1007
8,1008.png,1008
9,1009.jpg,1009


In [218]:
#menggabungkan test_df dengan test_file_df(dataframe nama file) sesuai dengan idnya
# kegiatan yang sama sebelumnya pada train_df
test_df = pd.merge(test_df, test_file_df, on='id', how='inner')

In [219]:
# memberikan alamat lengkap setiap file gambar sesuai dengan filename dalam dataframe
test_df['path'] = test_df['filename'].apply(lambda x: f'{test_file_path}{x}')

# menghapus columns yang sudah tidak dipakai lagi yaitu column filename
test_df.drop(columns=['filename'], inplace=True)

In [220]:
test_df.head(10)

,id,path
0,778,Penyisihan Hology Data Mining/test/778.png
1,779,Penyisihan Hology Data Mining/test/779.jpg
2,780,Penyisihan Hology Data Mining/test/780.jpg
3,781,Penyisihan Hology Data Mining/test/781.jpg
4,782,Penyisihan Hology Data Mining/test/782.jpg
5,783,Penyisihan Hology Data Mining/test/783.jpg
6,784,Penyisihan Hology Data Mining/test/784.jpg
7,785,Penyisihan Hology Data Mining/test/785.jpg
8,786,Penyisihan Hology Data Mining/test/786.jpg
9,787,Penyisihan Hology Data Mining/test/787.jpg


# Model test

In [225]:
THRESHOLD = 0.5

def make_predictions(model, test_df, image_size=(128, 128)):
    # Prepare a dictionary to store prediction results
    predictions = {
        'id': [],
        'jenis': [],
        'warna': []
    }
    
    # Iterate through the test DataFrame
    for idx, row in test_df.iterrows():
        img_id = row['id']
        img_path = row['path']
        
        # Load and preprocess the image
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Get the prediction
        pred = model.predict(img_array)[0]
        
        # Apply threshold to each label in the prediction
        binary_pred = (pred > THRESHOLD).astype(int)
        
        # susunan table Hoodie = 1, kaos = 0, biru = 2, hitam = 3, kuning = 1, merah = 0, putih =4
        # Determine `jenis` label
        jenis_pred = 1 if binary_pred[0] == 1 else (0 if binary_pred[1] == 1 else None)
        
        # Determine `warna` label based on color columns
        warna_pred = None
        if binary_pred[2] == 1:
            warna_pred = 2  # biru
        elif binary_pred[3] == 1:
            warna_pred = 3  # hitam
        elif binary_pred[4] == 1:
            warna_pred = 1  # kuning
        elif binary_pred[5] == 1:
            warna_pred = 0  # merah
        elif binary_pred[6] == 1:
            warna_pred = 4  # putih
        
        # Append results
        predictions['id'].append(img_id)
        predictions['jenis'].append(jenis_pred)
        predictions['warna'].append(warna_pred)
    
    # Convert predictions to DataFrame
    predictions_df = pd.DataFrame(predictions)
    
    return predictions_df

In [372]:
submission_df = make_predictions(model, test_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

In [229]:
submission_df.tail(20)

,id,jenis,warna
314,1092,0.0,2.0
315,1093,0.0,0.0
316,1094,1.0,1.0
317,1095,1.0,4.0
318,1096,0.0,4.0
319,1097,0.0,3.0
320,1098,0.0,1.0
321,1099,1.0,3.0
322,1100,1.0,4.0
323,1101,0.0,3.0


In [373]:
submission_df['jenis'] = submission_df['jenis'].fillna(0).astype(int)
submission_df['warna'] = submission_df['warna'].fillna(0).astype(int)


In [374]:
submission_df.to_csv('submission.csv', index=False)